In [26]:
import requests

From the command line, start the Flask application using development server on the Raspberry Pi.

    python3 qdotweb.py
    
The default address is http://qdot-pi.phas.ubc.ca:5000 . That page will show your computer name, the local time, and a list of available ports. We can also get a list of available ports and any \*IDN? resopnses from \instruments

In [27]:
r = requests.get('http://qdot-pi.phas.ubc.ca:5000/instruments')
jr = r.json()
print(jr['address'], jr['IDN'])

['ASRL/dev/ttyAMA0::INSTR', 'GPIB0::4::INSTR', 'url'] ['', 'Stanford_Research_Systems,SR830,s/n46843,ver1.07 \n', '']


There we see that at 'GPIB0::15::INSTR' (GPIB board 0, address 15) there is a Keithley 2400 Source Meter. Let's talk to it. To do that we will use the HTTP POST method to send some keyword arguments to the page.

In [28]:
# connect the instrument

r = requests.post('http://qdot-pi.phas.ubc.ca:5000/connect', 
                  data={'address':'GPIB0::4::INSTR'})
jr = r.json()
print(jr)

{'command': 'connect', 'errors': '', 'address': 'GPIB0::4::INSTR', 'data': 'CONNECTED'}


In [40]:
# read voltage

r = requests.post('http://qdot-pi.phas.ubc.ca:5000/query', 
                  data={'address':'GPIB0::4::INSTR', 
                        'command':'OUTP? 1'})
jr = r.json()
print(jr)

{'command': 'OUTP? 1', 'errors': '', 'address': 'GPIB0::4::INSTR', 'data': '0.193604\n'}


In [41]:
# close the instrument

r = requests.post('http://qdot-pi.phas.ubc.ca:5000/close', data={'address':'GPIB0::5::INSTR'})
jr = r.json()
print(jr)

{'command': 'close', 'errors': '', 'address': 'GPIB0::5::INSTR', 'data': 'CLOSED'}


Obviously, there is no need to print all of that data every time. A real program might look something like the following.

In [42]:
import time

# open connection
r = requests.post('http://qdot-pi.phas.ubc.ca:5000/connect', 
                  data={'address':'GPIB0::4::INSTR'})

N=100
tstart = time.time()
for i in range(N):
    r = requests.post('http://qdot-pi.phas.ubc.ca:5000/query', 
                  data={'address':'GPIB0::4::INSTR', 
                        'command':"OUTP? 1"})
    data = r.json()['data']
print('{} ms per request'.format((time.time()-tstart)*1000/N))

r = requests.post('http://qdot-pi.phas.ubc.ca:5000/close', 
                  data={'address':'GPIB0::4::INSTR'})

72.34816074371338 ms per request


That is compared to 145ms per request running the same commands locally on the raspberry pi.